<div dir="rtl" style="text-align: right;">
<h1>پیاده‌سازی کلاس Boids (رویکرد پایتونی)</h1>
   
<h2>پیاده‌سازی کلاس Boid (پایتون)</h2>
<p>از آنجایی که هر boid یک موجودیت خودمختار با چندین خاصیت مانند موقعیت و سرعت است، طبیعی به نظر می‌رسد که با نوشتن یک کلاس Boid شروع کنیم:</p>
</div>

In [16]:
import math
import random
from utils.vectow import vec2

class Boid:
    def __init__(self, x=0, y=0):
        self.position = vec2(x, y)
        angle = random.uniform(0, 2*math.pi)
        self.velocity = vec2(math.cos(angle), math.sin(angle))
        self.acceleration = vec2(0, 0)

<div dir="rtl" style="text-align: right;">
<p>شیء vec2 یک کلاس بسیار ساده است که تمامی عملیات برداری معمول با 2 مؤلفه را مدیریت می‌کند. این کلاس در کلاس اصلی Boid به ما در نوشتن کد کمک می‌کند. توجه داشته باشید که بسته‌های برداری متعددی در Python Package Index وجود دارد، اما برای چنین مثال ساده‌ای، این بسته‌ها بیش از حد پیچیده خواهند بود.</p>

<p>Boid یک مورد دشوار برای پایتون است زیرا هر boid با همسایگان محلی تعامل دارد. از آنجا که boids در حال حرکت هستند، در هر گام باید فاصله یک boid با هر boid دیگر که در شعاع تعامل قرار می‌گیرد محاسبه شود و سپس این فاصله‌ها مرتب شوند. روش نمونه‌وار نوشتن سه قانون به این صورت است:</p>
</div>

In [ ]:
def separation(self, boids):
  count = 0
  for other in boids:
    d = (self.position - other.position).length()
    if 0 < d < desired_separation:
      count += 1
      # ...
  if count > 0:
    # ...

def alignment(self, boids): # ...
def cohesion(self, boids): # ...

<div dir="rtl" style="text-align: right;">

<p>
برای کامل کردن می‌توانیم یک شیء 
Flock 
اضافه کنیم
</p>
</div>

In [ ]:
class Flock:
  def __init__(self, count=150):
    self.boids = []
    for i in range(count):
      boid = Boid()
      self.boids.append(boid)

  def run(self):
    for boid in self.boids:
      boid.run(self.boids)

<div dir="rtl" style="text-align: right;">
<h2>راه‌حل کامل</h2>
<p>با ترکیب تمامی منطق‌های فوق، پیاده‌سازی کامل Boids به صورت زیر است:</p>
</div>


In [20]:
# -----------------------------------------------------------------------------
# From Numpy to Python
# Copyright (2017) Nicolas P. Rougier - BSD license
# More information at https://github.com/rougier/numpy-book
# -----------------------------------------------------------------------------
import math
import random
from utils.vectow import vec2  # فرض شده که vec2 کلاس یا ماژول برداری تعریف شده‌ای است

class Boid:
    def __init__(self, x, y):
        self.acceleration = vec2(0, 0)
        angle = random.uniform(0, 2*math.pi)
        self.velocity = vec2(math.cos(angle), math.sin(angle))
        self.position = vec2(x, y)
        self.r = 2.0
        self.max_velocity = 2
        self.max_acceleration = 0.03

    def seek(self, target):
        desired = target - self.position
        desired = desired.normalized()
        desired *= self.max_velocity
        steer = desired - self.velocity
        steer = steer.limited(self.max_acceleration)
        return steer

    # پیاده‌سازی روش مرزها برای بازگشت پرندگان به فضای شبیه‌سازی
    def borders(self):
        x, y = self.position
        x = (x + self.width) % self.width
        y = (y + self.height) % self.height
        self.position = vec2(x, y)

    # جداسازی
    # این روش برای بررسی پرندگان نزدیک و دور شدن از آنها استفاده می‌شود
    def separate(self, boids):
        desired_separation = 25.0
        steer = vec2(0, 0)
        count = 0

        # بررسی فاصله پرندگان دیگر در سیستم
        for other in boids:
            d = (self.position - other.position).length()
            if 0 < d < desired_separation:
                # محاسبه بردار دور شدن از همسایه
                diff = self.position - other.position
                diff = diff.normalized()
                steer += diff / d  # وزن‌دهی بر اساس فاصله
                count += 1  # شمارش تعداد پرندگان نزدیک

        # محاسبه میانگین
        if count > 0:
            steer /= count

        # به شرطی که بردار بزرگتر از 0 باشد
        if steer.length() > 0:
            steer = steer.normalized()
            steer *= self.max_velocity
            steer -= self.velocity
            steer = steer.limited(self.max_acceleration)

        return steer

    # هم‌راستایی
    # برای هر پرنده نزدیک در سیستم، میانگین سرعت را محاسبه کنید
    def align(self, boids):
        neighbor_dist = 50
        sum = vec2(0, 0)
        count = 0
        for other in boids:
            d = (self.position - other.position).length()
            if 0 < d < neighbor_dist:
                sum += other.velocity
                count += 1

        if count > 0:
            sum /= count
            sum = sum.normalized()
            sum *= self.max_velocity
            steer = sum - self.velocity
            steer = steer.limited(self.max_acceleration)
            return steer
        else:
            return vec2(0, 0)

    # انسجام
    # برای میانگین موقعیت (مرکز) تمام پرندگان نزدیک، بردار هدایت به سمت آن موقعیت را محاسبه کنید
    def cohesion(self, boids):
        neighbor_dist = 50
        sum = vec2(0, 0)  # شروع با بردار خالی برای جمع‌آوری تمام موقعیت‌ها
        count = 0
        for other in boids:
            d = (self.position - other.position).length()
            if 0 < d < neighbor_dist:
                sum += other.position  # اضافه کردن موقعیت
                count += 1
        if count > 0:
            sum /= count
            return self.seek(sum)
        else:
            return vec2(0, 0)

    def flock(self, boids):
        sep = self.separate(boids)  # جداسازی
        ali = self.align(boids)  # هم‌راستایی
        coh = self.cohesion(boids)  # انسجام

        # وزن‌دهی اختیاری این نیروها
        sep *= 1.5
        ali *= 1.0
        coh *= 1.0

        # اضافه کردن بردار نیرو به شتاب
        self.acceleration += sep
        self.acceleration += ali
        self.acceleration += coh

    def update(self):
        # به‌روزرسانی سرعت
        self.velocity += self.acceleration
        # محدود کردن سرعت
        self.velocity = self.velocity.limited(self.max_velocity)
        self.position += self.velocity
        # بازنشانی شتاب به 0 در هر چرخه
        self.acceleration = vec2(0, 0)

    def run(self, boids):
        self.flock(boids)
        self.update()
        self.borders()


class Flock:
    def __init__(self, count=150, width=640, height=360):
        self.width = width
        self.height = height
        self.boids = []
        for i in range(count):
            boid = Boid(width / 2, height / 2)
            boid.width = width
            boid.height = height
            self.boids.append(boid)

    def run(self):
        for boid in self.boids:
            # انتقال کل لیست پرندگان به هر پرنده به صورت جداگانه
            boid.run(self.boids)


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

n = 50
flock = Flock(n)
P = np.zeros((n, 2))

def update(*args):
    flock.run()
    for i, boid in enumerate(flock.boids):
        P[i] = boid.position
    scatter.set_offsets(P)

fig = plt.figure(figsize=(8, 5))
ax = fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon=True)
scatter = ax.scatter(P[:, 0], P[:, 1],
                     s=30, facecolor="red", edgecolor="None", alpha=0.5)

ax.set_xlim(0, 640)
ax.set_ylim(0, 360)
ax.set_xticks([])
ax.set_yticks([])
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
anim = animation.FuncAnimation(fig, update, interval=10)
anim.save('statics/img/08_11_boids_vid.mp4', writer=writer)
plt.show()


/tmp/ipykernel_9639/612829923.py:176: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = animation.FuncAnimation(fig, update, interval=10)


<div dir="rtl" style="text-align: right;">
        <h2>معایب</h2>
        <p>با استفاده از این روش، ما می‌توانیم تا ۵۰ boid داشته باشیم تا زمانی که زمان محاسبه برای یک انیمیشن روان، بیش از حد کند شود. همانطور که ممکن است حدس زده باشید، می‌توانیم با استفاده از NumPy عملکرد بهتری داشته باشیم، اما ابتدا اجازه دهید به مشکل اصلی این پیاده‌سازی پایتون اشاره کنم.</p>
        <ul>
            <li>اگر به کد نگاه کنید، قطعاً متوجه می‌شوید که مقدار زیادی تکرار وجود دارد. دقیق‌تر بگوییم، ما از این واقعیت که فاصله اقلیدسی بازتابی است، بهره نمی‌بریم، یعنی ∣x−y∣=∣y−x∣.</li>
            <li>در این پیاده‌سازی ساده پایتون، هر قانون (تابع) n^2 فاصله را محاسبه می‌کند در حالی که n^2/2 کافی خواهد بود اگر به درستی ذخیره شود.</li>
            <li>علاوه بر این، هر قانون هر فاصله را بدون ذخیره نتیجه برای توابع دیگر دوباره محاسبه می‌کند. در نهایت، ما ۳n^2 فاصله را به جای n^2/2 محاسبه می‌کنیم.</li>
        </ul>

</div>